In [1]:
# YOU DO NOT NEED TO RUN THIS FILE.  It builds the df2_2001_2009 and df2_2010_201909 files.
# 20170519: fixed for Risk-based capital ratios (since 201503 for small banks)
# 20170405: altered to handle 201003 through 201612 (added code to handle partial current year)
# 20161212: added isFTE field (RIAD4150)
# process Call Rpt bulk data files  (major fixes needed to RCR schedule 20160507)
# WARNING: Requires that the Call Report bulk data files have been downloaded and unzipped
#          and placed in directories named C:/BankRisk/Bulk200103 (for March 2001 files as an example)
#          If there are any missing files, there will be error messages and the code will stop running

import numpy as np
import pandas as pd
from datetime import datetime
from time import strftime
#import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')
import os

os.chdir('C:/BankRisk')  # set directory for working data files here 

In [10]:
print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

2019-12-14 12:17:20


In [11]:
# altered to handle partial current year (with less than 4 quarters of data)

# use these rows for extract containing years 2001 - 2009:
#yst = 2001  # start year of bulk data to process
#yen = 2010  # end year to process (Note: python stops at 1 less than requested value!! if yen = 2010, will stop at 2009!!)
#last_mn = '12'  # usually set to '12' if all 4 qtrs of data available (if latest year is partial, put last month here)

# use these rows for extract containing years 2010 - 201612:
yst = 2010  # start year of bulk data to process
yen = 2020  # end year to process (Note: python stops at 1 less than requested value!! if yen = 2017, will stop at 2016!!)
last_mn = '09'  # usually set to '12' if all 4 qtrs of data available (if latest year is partial, put last month here)

yrs = np.arange(yst,yen)
mns = ['03', '06', '09', '12']
mds = ['0331', '0630', '0930','1231']
dmd = dict(zip(mns, mds))
dys = [31,30,30,31]
ddd = dict(zip(mns,dys))

yrmn_beg = str(min(yrs)) + '03'

for y in yrs:
  if y == (yen-1): mns = [x for x in mns if x <= last_mn] # shortens list of months if current year is partial
  for m in mns:
    asof = dmd[m] + str(y)
    yrmn = str(y) + m

    n = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Bulk POR "+asof+".txt", sep='\t', header=0, index_col=0)
    n.replace(to_replace='NATIONAL ASSOCIATION', value='NA', inplace=True, regex=True)

    nc = ['Cert','OCCchtr','OTSdckt','ABArt','Name','Address','City','State','Zip','FilTyp','SubmtDt']
    n.columns = nc

    # de-dup file:
    n = n.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')

    df = n[['Name','State','Cert']]

    ##########################################
    # process schedule RC:
    x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RC "+asof+".txt", sep='\t', header=0, index_col=0)
    x = x.ix[x.index[1:]]  # removes first data row containing col descriptions
    x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')

    #x.drop('RCON8678', axis=1, inplace=True)  # remove fiscal year-end date ("12-31")
    x = x.apply(pd.to_numeric, errors='ignore')
    x = x.fillna(0)

    # process raw data from schedule:
    df['eTotAst'] = x.apply(lambda x: max(x['RCON2170'],x['RCFD2170']),axis=1)
    df['eTotLn'] = x.apply(lambda x: max(x['RCON5369'],x['RCFD5369'])+max(x['RCONB528'],x['RCFDB528']),axis=1)
    df['eALLL'] = x.apply(lambda x: max(x['RCON3123'],x['RCFD3123']), axis=1)
    df['eOREO'] = x.apply(lambda x: max(x['RCON2150'],x['RCFD2150']), axis=1)

    try:
      df['eGW'] = x.apply(lambda x: max(x['RCON3163'],x['RCFD3163']), axis=1)
      df['OthIntang'] = x.apply(lambda x: max(x['RCON0426'],x['RCFD0426']), axis=1)
      df['TotIntang'] = 0
    except:
      df['eGW'] = 0
      df['OthIntang'] = 0
      df['TotIntang'] = x.apply(lambda x: max(x['RCON2143'],x['RCFD2143']), axis=1)
    
    try:
      df['eTotEq'] = x.apply(lambda x: max(x['RCONG105'],x['RCFDG105']), axis=1)
    except:
      df['eTotEq'] = x.apply(lambda x: max(x['RCON3210'],x['RCFD3210']), axis=1)

    df['eDDep'] = x['RCON2200']
    df['eDDepNIB'] = x['RCON6631']
    df['eDDepIB'] = x['RCON6636']
    df['eFDep'] = x['RCFN2200']
    df['eFDepNIB'] = x['RCFN6631']
    df['eFDepIB'] = x['RCFN6636']
    df['eDDA'] = df['eDDepNIB']+df['eFDepNIB']

    
    ###################################################################
    # process schedule RCM: (memoranda) 
    #if 1==1:
    x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCM "+asof+".txt", sep='\t', header=0, index_col=0)
    x= x.ix[x.index[1:]]  # removes first data row containing col descriptions
    x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
    
    x = x.apply(pd.to_numeric, errors='ignore')
    x=x.fillna(0)

    # process raw data from schedule:
    #df['eGW'] = x.apply(lambda x: max(x['RCON3163'],x['RCFD3163']), axis=1)
    df['MSR'] = x.apply(lambda x: max(x['RCON3164'],x['RCFD3164']), axis=1)

    try:
      df['OthIntang'] = x.apply(lambda x: max(x['RCONJF76'],x['RCFDJF76']), axis=1)
      df['TotIntang'] = x.apply(lambda x: max(x['RCON2143'],x['RCFD2143']), axis=1)
    except:
      df['OthIntang'] = 0
      df['TotIntang'] = 0

    df['eGW'] = df.apply(lambda x: max(x['eGW'],x['TotIntang']-x['MSR']-x['OthIntang']),axis=1)
    df['TotIntang'] = df.apply(lambda x: max(x['TotIntang'],x['eGW']+x['MSR']+x['OthIntang']),axis=1)
    df['eTangEq'] = df.apply(lambda x:x['eTotEq'] - x['TotIntang'],axis=1)
    
    ################################################
    # process schedule RCK: qtrly average balances:
    x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCK "+asof+".txt", sep='\t', header=0, index_col=0)
    x= x.ix[x.index[1:]]  # removes first data row containing col descriptions
    x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')

    x = x.apply(pd.to_numeric, errors='ignore')
    x=x.fillna(0)

    df['aTotAst'] = x.apply(lambda x: max(x['RCON3368'],x['RCFD3368']),axis=1)
    df['aDLn'] = x['RCON3360']

    try:
      df['aSFR'] = x['RCON3465']
      df['aRE'] = x['RCON3466']
      df['aTotRE'] = x['RCON3465'] + x['RCON3466']
    except:
      df['aSFR'] = 0
      df['aRE'] = 0
      df['aTotRE'] = x['RCON3385']

    df['aAg'] = x['RCON3386']
    df['aCI'] = x['RCON3387']
    df['aCard'] = x['RCONB561']
    df['aOthCsmr'] = x['RCONB562']
    df['aFLn'] = x['RCFN3360']
    df['aLeas'] = x.apply(lambda x: max(x['RCON3484'],x['RCFD3484']), axis=1)
    df['aTrad'] = x.apply(lambda x: max(x['RCON3401'],x['RCFD3401']), axis=1)

    df['aDFIB'] = x.apply(lambda x: max(x['RCON3381'],x['RCFD3381']), axis=1)
    df['aTreas'] = x.apply(lambda x: max(x['RCONB558'],x['RCFDB558']), axis=1)
    df['aMBS'] = x.apply(lambda x: max(x['RCONB559'],x['RCFDB559']), axis=1)
    df['aOSec'] = x.apply(lambda x: max(x['RCONB560'],x['RCFDB560']), axis=1)
    df['aFFS'] = x.apply(lambda x: max(x['RCON3365'],x['RCFD3365']), axis=1)
    
    df['aTotLn'] = x['RCON3360'] + x['RCFN3360']
    df['aTotLnLs'] = df['aTotLn'] + df['aLeas']  # 20160320
    df['aOthLnLs'] = df['aTotLnLs'] - df['aCI'] - df['aTotRE'] - df['aCard'] - df['aOthCsmr']  # 20160320
    
    df['aTotSec'] = df['aTreas'] + df['aMBS'] + df['aOSec']
    df['aOAstIB'] = df['aFFS'] + df['aDFIB'] +df['aTrad']
    df['aEarnAst'] = df['aTotLn']+df['aTrad']+df['aDFIB']+df['aTreas']+df['aMBS']+\
                     df['aOSec']+df['aFFS']+df['aLeas']

    df['aNOW'] = x['RCON3485']
    df['aSav'] = x['RCONB563']
    try:
      df['aCDun'] = x['RCONA529']
      df['aCDov'] = x['RCONA514']
    except:
      df['aCDun'] = x['RCONHK16']
      df['aCDov'] = x['RCONHK17']

    df['aDDepIB'] = df['aNOW'] + df['aSav'] + df['aCDun'] + df['aCDov']
    df['aFDepIB'] = x['RCFN3404']
    df['aFFP'] = x.apply(lambda x: max(x['RCON3353'],x['RCFD3353']), axis=1)
    df['aBorr'] = x.apply(lambda x: max(x['RCON3355'],x['RCFD3355']), axis=1)
    df['aDepIB'] = df['aDDepIB']+df['aFDepIB']
    df['aOLbIB'] = df['aFFP'] + df['aBorr']
    
    ########################################################
    # process schedule RCCI: ending loan balances:
    x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCCI "+asof+".txt", sep='\t', header=0, index_col=0)
    x = x.ix[x.index[1:]]  # removes first data row containing col descriptions
    x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')

    x = x.apply(pd.to_numeric, errors='ignore')
    x = x.fillna(0)

    # process raw data from schedule RC-CI:
    df['eTotLn'] = x.apply(lambda x: max(x['RCON2122'],x['RCFD2122']),axis=1)
    df['eDLn'] = x['RCON2122']

    try:
      df['eCxSFR'] = x['RCONF158']
      df['eCxOth'] = x['RCONF159']
      df['eCx'] = x['RCONF158'] + x['RCONF159']
    except:
      df['eCxSFR'] = 0
      df['eCxOth'] = 0
      df['eCx'] = x['RCON1415']

    df['eFarm'] = x['RCON1420']
    df['eSfRev'] = x['RCON1797']
    df['eSfFst'] = x['RCON5367']
    df['eSfSec'] = x['RCON5368']
#    df['eSfTD'] = x['RCON1797'] + x['RCON5367']  # added 6/10/15. Needed for loss vol calcs
    df['eSfTD'] = x['RCON5367'] + x['RCON5368']  # corrected 8/18/15 from prior line
    df['eMult'] = x['RCON1460']

    try:
      df['eOwOc'] = x['RCONF160']
      df['eOthCRE'] = x['RCONF161']
      df['eCRE'] = x['RCONF160'] + x['RCONF161']
    except:
      df['eOwOc'] = 0
      df['eOthCRE'] = 0
      df['eCRE'] = x['RCON1480'] 

    df['eCREtot'] = df['eCx'] + df['eMult'] + df['eCRE']
    df['eFinInst'] = x.apply(lambda x: max(x['RCON1288'],(x['RCFDB532']+x['RCFDB533']+\
                            x['RCFDB534']+x['RCFDB536']+x['RCFDB537'])),axis=1)
    df['eAg'] = x.apply(lambda x: max(x['RCON1590'],x['RCFD1590']),axis=1)
    df['eCIdom'] = x.apply(lambda x: max(x['RCON1763'],x['RCFD1763']),axis=1)
    df['eCIfor'] = x.apply(lambda x: max(x['RCON1764'],x['RCFD1764']),axis=1)
    df['eCard'] = x.apply(lambda x: max(x['RCONB538'],x['RCFDB538']),axis=1)
    df['eOthRev'] = x.apply(lambda x: max(x['RCONB539'],x['RCFDB539']),axis=1)
    try:
      df['eAuto'] = x.apply(lambda x: max(x['RCONK137'],x['RCFDK137']),axis=1)
      df['eOthCsmr'] = x.apply(lambda x: max(x['RCONK207'],x['RCFDK207']),axis=1)
    except:
      df['eAuto'] = 0
      df['eOthCsmr'] = x.apply(lambda x: max(x['RCON2011'],x['RCFD2011']),axis=1)

    df['eForGovt'] = x.apply(lambda x: max(x['RCON2081'],x['RCFD2081']),axis=1)
    #df['eOthLn'] = x.apply(lambda x: max((x['RCONJ454']+x['RCONJ464']),(x['RCFD2107']+x['RCFD1563'])),axis=1)

    try:
      df['eLeas'] = x.apply(lambda x: max(x['RCON2165'],(x['RCFDF162']+x['RCFDF163'])),axis=1)
    except:
      df['eLeas'] = x.apply(lambda x: max(x['RCON2165'],(x['RCFD2182']+x['RCFD2183'])),axis=1)

    df['eUnearn'] = x.apply(lambda x: max(x['RCON2123'],x['RCFD2123']),axis=1)

    ###########################################################
    # process schedule RCN: non-accrual loans (need both parts)
    try:
      x1 = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCN "+asof+"(1 of 2).txt", sep='\t', header=0, index_col=0)
      x1= x1.ix[x1.index[1:]]  # removes first data row containing col descriptions
      x1 = x1.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
  
      x2 = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCN "+asof+"(2 of 2).txt", sep='\t', header=0, index_col=0)
      x2= x2.ix[x2.index[1:]]  # removes first data row containing col descriptions
      x2 = x2.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')

      x = x1.merge(x2, how='inner', left_index=True, right_index=True)
    except:
      x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCN "+asof+".txt", sep='\t', header=0, index_col=0)
      x = x.ix[x.index[1:]]  # removes first data row containing col descriptions
      x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
    x = x.apply(pd.to_numeric, errors='ignore')
    x = x.fillna(0)

    # process nonaccrual loans from schedule RC-N:

    try:
      df['nCxSFR'] = x['RCONF176']
      df['nCxOth'] = x['RCONF177']
      df['nCx'] = x['RCONF176'] + x['RCONF177']
    except:
      df['nCxSFR'] = 0
      df['nCxOth'] = 0
      df['nCx'] = x['RCON3492']

    df['nFarm'] = x['RCON3495']
    df['nSfRev'] = x['RCON5400']

    try:
      df['nSfFst'] = x['RCONC229']
      df['nSfSec'] = x['RCONC230']
      df['nSfTD'] = x['RCONC229'] + x['RCONC230']
    except:
      df['nSfFst'] = 0
      df['nSfSec'] = 0
      df['nSfTD'] = x['RCON5403']

    df['nMult'] = x['RCON3501']

    try:
      df['nOwOc'] = x['RCONF182']
      df['nOthCRE'] = x['RCONF183']
      df['nCRE'] = x['RCONF182'] + x['RCONF183']
    except:
      df['nOwOc'] = 0
      df['nOthCRE'] = 0
      df['nCRE'] = x['RCON3504']

    df['nCREtot'] = df['nCx'] + df['nMult'] + df['nCRE']
    df['nREfor'] = x['RCFNB574']
    df['nFinInst'] = x.apply(lambda x: max(x['RCONB836'],(x['RCFD5379']+x['RCFD5382'])),axis=1)
    df['nAg'] = x['RCFD1583']
    df['nCIdom'] = x.apply(lambda x: max(x['RCON1608'],x['RCFD1253']),axis=1)
    df['nCIfor'] = x['RCFD1256']
    df['nCard'] = x.apply(lambda x: max(x['RCONB577'],x['RCFDB577']),axis=1)
    try:
      df['nAuto'] = x.apply(lambda x: max(x['RCONK215'],x['RCFDK215']),axis=1)
      df['nOthCsmr'] = x.apply(lambda x: max(x['RCONK218'],x['RCFDK218']),axis=1)
    except:
      df['nAuto'] = 0
      df['nOthCsmr'] = x.apply(lambda x: max(x['RCONB580'],x['RCFDB580']),axis=1)

    try:
      df['nForGovt'] = x.apply(lambda x: max(x['RCON5391'],x['RCFD5391']),axis=1)
      df['nOthLn'] = x.apply(lambda x: max(x['RCON5461'],x['RCFD5461']),axis=1)
    except:
      df['nForGovt'] = x['RCFD5391']
      df['nOthLn'] = x['RCFD5461']

    try:
      df['nLeas'] = x.apply(lambda x: max(x['RCON1228'],(x['RCFDF168']+x['RCFDF171'])),axis=1)
    except:
      df['nLeas'] = x.apply(lambda x: max(x['RCON1228'],(x['RCFD1259']+x['RCFD1791'])),axis=1)

    df['nTotLn'] = df['nCx']+df['nFarm']+df['nSfRev']+df['nSfTD']+df['nMult']+\
                   df['nCRE']+df['nREfor']+df['nFinInst']+df['nAg']+df['nCIdom']+df['nCIfor']+\
                   df['nCard']+df['nAuto']+df['nOthCsmr']+df['nForGovt']+df['nOthLn']+df['nLeas']


    ###################################################################
    # process schedule RCE: (misc items)
    x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCE "+asof+".txt", sep='\t', header=0, index_col=0)
    x= x.ix[x.index[1:]]  # removes first data row containing col descriptions
    x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
    
    x = x.apply(pd.to_numeric, errors='ignore')
    x=x.fillna(0)

    # process raw data from schedule:

    df['eBrok'] = x['RCON2365']
    df['eMMDA'] = x['RCON6810']
    df['eOthSav'] = x['RCON0352']
    try:
      df['eCDov250'] = x['RCONJ474']
    except:
      df['eCDov250'] = 0
    
    ###################################################################
    # process schedule RCO: (more misc items)
    try:
      x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCO "+asof+".txt", sep='\t', header=0, index_col=0)
    except:
      x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCO "+asof+"(1 of 2).txt", sep='\t', header=0, index_col=0)  

    x = x.ix[x.index[1:]]  # removes first data row containing col descriptions
    x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
    x = x.apply(pd.to_numeric, errors='ignore')
    x = x.fillna(0)

    try:
      df['eDepov250'] = x['RCONF051']
      df['eIRAov250'] = x['RCONF047']
      df['eUninsDep'] = x['RCON5597']
    except:
      df['eDepov250'] = 0
      df['eIRAov250'] = 0
      df['eUninsDep'] = 0

    #####################################################################
    # process schedule RCR: regulatory capital
    try:
      x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCR "+asof+"(1 of 2).txt", sep='\t', header=0, index_col=0)
      x = x.ix[x.index[1:]]  # removes first data row containing col descriptions
      x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
      x2 = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCR "+asof+"(2 of 2).txt", sep='\t', header=0, index_col=0)
      x2 = x2.ix[x2.index[1:]]  # removes first data row containing col descriptions
      x2 = x2.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
      x = x.merge(x2, how='inner', left_index=True, right_index=True)
    except:
      try:
        x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCRIA "+asof+".txt", sep='\t', header=0, index_col=0)
        x = x.ix[x.index[1:]]  # removes first data row containing col descriptions
        x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
        x2 = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCRIB "+asof+".txt", sep='\t', header=0, index_col=0)
        x2 = x2.ix[x2.index[1:]]  # removes first data row containing col descriptions
        x2 = x2.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
        x3 = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCRII "+asof+"(1 of 2).txt", sep='\t', header=0, index_col=0)
        x3 = x3.ix[x3.index[1:]]  # removes first data row containing col descriptions
        x3 = x3.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
        x = x.merge(x2, how='inner', left_index=True, right_index=True)
        x = x.merge(x3, how='inner', left_index=True, right_index=True)
      except:
        try:
          x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCRI "+asof+".txt", sep='\t', header=0, index_col=0)
          x = x.ix[x.index[1:]]  # removes first data row containing col descriptions
          x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
          x3 = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCRII "+asof+"(1 of 3).txt", sep='\t', header=0, index_col=0)
          x3 = x3.ix[x3.index[1:]]  # removes first data row containing col descriptions
          x3 = x3.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
          x = x.merge(x3, how='inner', left_index=True, right_index=True)
        except:
          x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCRI "+asof+".txt", sep='\t', header=0, index_col=0)
          x = x.ix[x.index[1:]]  # removes first data row containing col descriptions
          x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
          x3 = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCRII "+asof+"(1 of 4).txt", sep='\t', header=0, index_col=0)
          x3 = x3.ix[x3.index[1:]]  # removes first data row containing col descriptions
          x3 = x3.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
          x = x.merge(x3, how='inner', left_index=True, right_index=True)

    x = x.apply(lambda z: pd.to_numeric(z, errors='coerce'))  ## 20160507
    #x = x.apply(pd.to_numeric, errors='ignore')  # deprecated with v0.18
    x = x.fillna(0)
    if 'RCON7206' not in x.columns:  ## 20170519: these cols were removed by FDIC in 2015
      x['RCON7206'] = x['RCOA7206'] # eT1rbc
      x['RCON7205'] = x['RCOA7205'] # eTotrbc
      x['RCON8274'] = x['RCOA8274'] # eT1Cap
      x['RCON3792'] = x['RCOA3792'] # eTotCap
      x['RCONA223'] = x['RCOAA223'] # eTotRWA
      x['RCONL138'] = x['RCOAA224'] # eTotAstLev

    try:
      df['eT1rbc'] = x.apply(lambda x: max(x['RCON7206'],x['RCFA7206'],x['RCFW7206']), axis=1)
      df['eTotrbc'] = x.apply(lambda x: max(x['RCON7205'],x['RCFA7205'],x['RCFW7205']), axis=1)
    except:
      df['eT1rbc'] = x.apply(lambda x: max(x['RCON7206'],x['RCFD7206'],x['RCFD7274']), axis=1)
      df['eTotrbc'] = x.apply(lambda x: max(x['RCON7205'],x['RCFD7205'],x['RCFD7275']), axis=1)

    try:
      df['eT1Cap'] = x.apply(lambda x: max(x['RCON8274'],x['RCFA8274']), axis=1)
      df['eTotCap'] = x.apply(lambda x: max(x['RCON3792'],x['RCFA3792']), axis=1)
      df['eTotRWA'] = x.apply(lambda x: max(x['RCONA223'],x['RCFAA223']), axis=1)
    except:
      df['eT1Cap'] = x.apply(lambda x: max(x['RCON8274'],x['RCFD8274']), axis=1)
      df['eTotCap'] = x.apply(lambda x: max(x['RCON3792'],x['RCFD3792']), axis=1)
      df['eTotRWA'] = x.apply(lambda x: max(x['RCONA223'],x['RCFDA223']), axis=1)

    try:
      df['eTotAstLev'] = x.apply(lambda x: max(x['RCONL138'],x['RCFAA224']), axis=1)
    except:
      try:
        df['eTotAstLev'] = x.apply(lambda x: max(x['RCONL138'],x['RCFDL138']), axis=1)
      except:
        df['eTotAstLev'] = x.apply(lambda x: max(x['RCONA224'],x['RCFDA224']), axis=1)


    df['eT1rbc'] = df.apply(lambda z: z['eT1rbc'] if z['eT1rbc']>0 else z['eT1Cap']/(z['eTotRWA']+.0001), axis=1)
    df['eTotrbc'] = df.apply(lambda z: z['eTotrbc'] if z['eTotrbc']>0 else z['eTotCap']/(z['eTotRWA']+.0001), axis=1)

    #####################################################################
    # process schedule RI: income statement
    x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RI "+asof+".txt", sep='\t', header=0, index_col=0)
    x= x.ix[x.index[1:]]  # removes first data row containing col descriptions

    #x.drop('RIADA530', axis=1, inplace=True)
    x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
    x = x.apply(pd.to_numeric, errors='ignore')
    x=x.fillna(0)

    try:
      df['iiSFR'] = x['RIAD4435']
      df['iiOthRE'] = x['RIAD4436']
      df['iiRE'] = x['RIAD4435'] + x['RIAD4436']
    except:
      df['iiSFR'] = 0
      df['iiOthRE'] = 0
      df['iiRE'] = x['RIAD4011']

    df['iiAg'] = x['RIAD4024']
    df['iiCI'] = x['RIAD4012']
    df['iiCard'] = x['RIADB485']
    df['iiOthCsmr'] = x['RIADB486']
    df['iiForGovt'] = x['RIAD4056']
    df['iiOthDomLn'] = x.apply(lambda x: max(x['RIAD4058'],x['RIADB487']), axis=1)
    df['iiFLn'] = x['RIAD4059']
    df['iiTotLn'] = x['RIAD4010']
    df['iiLeas'] = x['RIAD4065']
    df['iiDFIB'] = x['RIAD4115']
    df['iiTreas'] = x['RIADB488']
    df['iiMBS'] = x['RIADB489']
    df['iiOSec'] =x['RIAD4060']
    df['iiTrad'] =x['RIAD4069']
    df['iiFFS'] = x['RIAD4020']
    df['iiOthII'] = x['RIAD4518']
    df['iiTotLnLs'] = df['iiTotLn']+df['iiLeas']
    df['iiOthLnLs'] = df['iiTotLnLs'] - df['iiRE'] - df['iiCI'] - df['iiCard'] - df['iiOthCsmr'] # 20160320
    df['iiTotSec'] = df['iiTreas']+df['iiMBS']+df['iiOSec']
    df['iiOAstIB'] = df['iiDFIB']+df['iiTrad']+df['iiFFS']+df['iiOthII']
    df['iiTotII_pc'] = df['iiTotLnLs']+df['iiTotSec']+df['iiOAstIB']
    df['iiTotII'] = x['RIAD4107']

    df['ieNOW'] = x['RIAD4508']
    df['ieSav'] = x['RIAD0093']
    try:
      df['ieCDun'] = x['RIADA518']
      df['ieCDov'] = x['RIADA517']
    except:
      df['ieCDun'] = x['RIADHK03']
      df['ieCDov'] = x['RIADHK04']

    df['ieFDepIB'] = x['RIAD4172']
    df['ieFFP'] = x['RIAD4180']
    df['ieOthBorr'] = x['RIAD4185']
    df['ieNote'] = x['RIAD4200']
    df['ieDepIB'] = df['ieNOW'] + df['ieSav'] + df['ieCDun'] + df['ieCDov'] + df['ieFDepIB']
    df['ieOLbIB'] = df['ieFFP'] + df['ieOthBorr'] + df['ieNote']
    df['ieDDA'] = 0
    df['ieTotIE_pc'] = df['ieDepIB'] + df['ieOLbIB']
    df['ieTotIE'] = x['RIAD4073']
    df['iiNetII'] = x['RIAD4074']
    
    try:
      df['isIntangExp'] = x['RIADC216'] + x['RIADC232']
    except:
      df['isIntangExp'] = x['RIAD4531']

    try:
      df['isProv'] = x['RIAD4230'] 
    except:
      df['isProv'] = x['RIADJJ33']        ##  as of 2019-Mar

    df['isTotNII'] = x['RIAD4079']
    df['isTotNIE'] = x['RIAD4093']
    df['isPreTax'] = x['RIAD4301']
    df['isNetInc'] = x['RIAD4340']

    ## added 10/31/15:  gain/loss items:
    df['isOREOgl'] = x['RIAD5415']
    df['isLnLsgl'] = x['RIAD5416']
    df['isOthAstgl'] = x['RIADB496']
    df['isHTMgl'] = x['RIAD3521']
    df['isAFSgl'] = x['RIAD3196']
    
    ## added isFTE on 12/12/16:
    df['isFTE'] = x['RIAD4150']
    
    #########################################################################
    # process schedule RIBI: net charge-offs
    x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RIBI "+asof+".txt", sep='\t', header=0, index_col=0)
    x= x.ix[x.index[1:]]  # removes first data row containing col descriptions

    x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
    #x.drop('RIADA530', axis=1, inplace=True)
    x=x.astype(float)
    x=x.fillna(0)

    df['ncoTotLn'] = x['RIAD4635'] - x['RIAD4605']

    try:
      df['ncoCxSFR'] = x['RIADC891'] - x['RIADC892']
      df['ncoCxOth'] = x['RIADC893'] - x['RIADC894']
      df['ncoCx'] = x['RIADC891'] - x['RIADC892'] + x['RIADC893'] - x['RIADC894']
    except:
      df['ncoCxSFR'] = 0
      df['ncoCxOth'] = 0
      df['ncoCx'] = x['RIAD3582'] - x['RIAD3583']

    df['ncoFarm'] = x['RIAD3584'] - x['RIAD3585']
    df['ncoSfRev'] = x['RIAD5411'] - x['RIAD5412']

    try:
      df['ncoSfFst'] = x['RIADC234'] - x['RIADC217']
      df['ncoSfSec'] = x['RIADC235'] - x['RIADC218']
      df['ncoSfTD'] = x['RIADC234'] - x['RIADC217'] + x['RIADC235'] - x['RIADC218']
    except:
      df['ncoSfFst'] = 0
      df['ncoSfSec'] = 0
      df['ncoSfTD'] = x['RIAD5413'] - x['RIAD5414'] 

    df['ncoMult'] = x['RIAD3588'] - x['RIAD3589']

    try:
      df['ncoOwOc'] = x['RIADC895'] - x['RIADC896']
      df['ncoOthCRE'] = x['RIADC897'] - x['RIADC898']
      df['ncoCRE'] = x['RIADC895'] - x['RIADC896'] + x['RIADC897'] - x['RIADC898']
    except:
      df['ncoOwOc'] = 0
      df['ncoOthCRE'] = 0
      df['ncoCRE'] = x['RIAD3590'] - x['RIAD3591']

    df['ncoCREtot'] = df['ncoCx'] + df['ncoMult'] + df['ncoCRE']
    df['ncoREfor'] = x['RIADB512'] - x['RIADB513']
    try:
      df['ncoFIdom'] = x.apply(lambda x: max((x['RIAD4481']-x['RIAD4654']),x['RIAD4653']) -\
                               max((x['RIAD4482']-x['RIAD4664']),x['RIAD4663']),axis=1)
      df['ncoFIfor'] = x['RIAD4654'] - x['RIAD4664']
    except:
      df['ncoFIdom'] = 0
      df['ncoFIfor'] = 0
        
    df['ncoAg'] = x['RIAD4655'] - x['RIAD4665']
    df['ncoCIdom'] = x.apply(lambda x: max((x['RIAD4638']-x['RIAD4646']),x['RIAD4645']) -\
                             max((x['RIAD4608']-x['RIAD4618']),x['RIAD4617']),axis=1)
    df['ncoCIfor'] = x['RIAD4646'] - x['RIAD4618']
    df['ncoCard'] = x['RIADB514'] - x['RIADB515']
    try:
      df['ncoAuto'] = x['RIADK129'] - x['RIADK133']
      df['ncoOthCsmr'] = x['RIADK205'] - x['RIADK206']
    except:
      df['ncoAuto'] = 0
      df['ncoOthCsmr'] = x['RIADB516'] - x['RIADB517']

    df['ncoForGovt'] = x['RIAD4643'] - x['RIAD4627']
    # ncoOthLn contains ncoAg for small banks:  must be removed:
    def OthLnCalc(x):
      adj = 0
      if x['RIAD4645'] == 0:
        adj = x['RIAD4655'] - x['RIAD4665']
      return x['RIAD4644'] - x['RIAD4628'] - adj

    df['ncoOthLn'] = x.apply(OthLnCalc, axis = 1)

    try:
      df['ncoLeas'] = x.apply(lambda x: max(x['RIAD4266'],(x['RIADF185']+x['RIADC880'])) -\
                              max(x['RIAD4267'],(x['RIADF187']+x['RIADF188'])),axis=1)
    except:
      df['ncoLeas'] = x.apply(lambda x: max(x['RIAD4266'],(x['RIAD4658']+x['RIAD4659'])) -\
                              max(x['RIAD4267'],(x['RIAD4668']+x['RIAD4669'])),axis=1)

    #################################################################
    # add the "as of" date column and change the index to be IDRSSD (ref #) and as-of date:
    df['AsOf'] = datetime(y,int(m),ddd[m])
    df = df.reset_index()
    df = df.set_index(['IDRSSD','AsOf'])
    #df.head()
    if yrmn == yrmn_beg:
      df2 = df
    else:
      df2 = pd.concat([df2, df])
    #################################################################
  print(y, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

2010 2019-12-14 12:18:10
2011 2019-12-14 12:19:01
2012 2019-12-14 12:19:55
2013 2019-12-14 12:20:49
2014 2019-12-14 12:21:40
2015 2019-12-14 12:22:29
2016 2019-12-14 12:23:17
2017 2019-12-14 12:24:00
2018 2019-12-14 12:24:42
2019 2019-12-14 12:25:13


In [12]:
df2.to_csv("df2_" + str(yst) + "_" + str(yen-1) + last_mn + ".csv")
len(df2) # number of data rows # 291793 (2001-200912), 189394 (2010-201612), 206930 (2010-201709), 212651 (2010-201712)
# 223900 (2010-201806), 234884 (2010-201812), 240295 (201903), 250952 (201909)

250952

In [13]:
"df2_" + str(yst) + "_" + str(yen-1) + last_mn + ".csv"  # name of output file (Note: end year is 1 less than yen value!!)

'df2_2010_201909.csv'

In [14]:
df2.tail(2)

Name State   Cert  eTotAst  eTotLn  eALLL  eOREO  \
IDRSSD  AsOf                                                                   
5401273 2019-09-30   TANDEM BANK    GA  59178  20774.0     0.0    0.0    0.0   
5412457 2019-09-30  LEXICON BANK    NV  59117  23626.0   227.0    0.0    0.0   

                    eGW  OthIntang  TotIntang  ...  ncoFIfor  ncoAg  ncoCIdom  \
IDRSSD  AsOf                                   ...                              
5401273 2019-09-30  0.0        0.0        0.0  ...       0.0    0.0       0.0   
5412457 2019-09-30  0.0        0.0        0.0  ...       0.0    0.0       0.0   

                    ncoCIfor  ncoCard  ncoAuto  ncoOthCsmr  ncoForGovt  \
IDRSSD  AsOf                                                             
5401273 2019-09-30       0.0      0.0      0.0         0.0         0.0   
5412457 2019-09-30       0.0      0.0      0.0         0.0         0.0   

                    ncoOthLn  ncoLeas  
IDRSSD  AsOf                           
5401273 2019-09-30       0.0      0.0  
5412457 2019-09-30       0.0      0.0  

[2 rows x 193 columns]

In [15]:
df2a = df2.reset_index()
df2a.head(3)

IDRSSD       AsOf                              Name State   Cert   eTotAst  \
0      37 2010-03-31            BANK OF HANCOCK COUNTY    GA  10057   81828.0   
1     242 2010-03-31  FIRST COMMUNITY BANK XENIA-FLORA    IL   3850   32421.0   
2     279 2010-03-31       MINEOLA COMMUNITY BANK, SSB    TX  28868  158209.0   

    eTotLn  eALLL  eOREO  eGW  ...  ncoFIfor  ncoAg  ncoCIdom  ncoCIfor  \
0  43966.0  963.0   50.0  0.0  ...       0.0    0.0       0.0       0.0   
1  20904.0  174.0    0.0  0.0  ...       0.0    0.0       0.0       0.0   
2  72833.0  569.0   94.0  0.0  ...       0.0    0.0       0.0       0.0   

   ncoCard  ncoAuto  ncoOthCsmr  ncoForGovt  ncoOthLn  ncoLeas  
0      0.0      0.0        33.0         0.0       0.0      0.0  
1      0.0      0.0        -2.0         0.0       0.0      0.0  
2      0.0      0.0        12.0         0.0       0.0      0.0  

[3 rows x 195 columns]

In [16]:
df2a[['AsOf','IDRSSD']].groupby('AsOf').count()

IDRSSD
AsOf              
2010-03-31    7250
2010-06-30    7152
2010-09-30    7094
2010-12-31    6999
2011-03-31    6923
2011-06-30    6876
2011-09-30    6820
2011-12-31    6789
2012-03-31    7377
2012-06-30    7314
2012-09-30    7249
2012-12-31    7150
2013-03-31    7085
2013-06-30    7005
2013-09-30    6955
2013-12-31    6877
2014-03-31    6794
2014-06-30    6722
2014-09-30    6651
2014-12-31    6570
2015-03-31    6480
2015-06-30    6408
2015-09-30    6328
2015-12-31    6238
2016-03-31    6176
2016-06-30    6112
2016-09-30    6034
2016-12-31    5966
2017-03-31    5908
2017-06-30    5839
2017-09-30    5789
2017-12-31    5721
2018-03-31    5657
2018-06-30    5593
2018-09-30    5527
2018-12-31    5456
2019-03-31    5411
2019-06-30    5352
2019-09-30    5305

In [17]:
# check for rbc elements
#      x['RCON7206'] = x['RCOA7206'] # eT1rbc
#      x['RCON7205'] = x['RCOA7205'] # eTotrbc
#      x['RCON8274'] = x['RCOA8274'] # eT1Cap
#      x['RCON3792'] = x['RCOA3792'] # eTotCap
#      x['RCONA223'] = x['RCOAA223'] # eTotRWA
#      x['RCONL138'] = x['RCOAA224'] # eTotAstLev
df2a[df2a['IDRSSD']==37][['AsOf','eT1Cap','eTotCap','eT1rbc','eTotrbc','eTotRWA','eTotAstLev']]

AsOf   eT1Cap  eTotCap    eT1rbc   eTotrbc  eTotRWA  eTotAstLev
0      2010-03-31  16665.0  17249.0  0.359600  0.372200  46342.0     82587.0
7250   2010-06-30  16700.0  17296.0  0.353000  0.365600  47314.0     82131.0
14402  2010-09-30  16913.0  17511.0  0.355500  0.368000  47581.0     82132.0
21496  2010-12-31  16892.0  17508.0  0.345300  0.357900  48915.0     83108.0
28495  2011-03-31  17012.0  17582.0  0.376600  0.389200  45177.0     84166.0
35418  2011-06-30  17025.0  17591.0  0.379700  0.392300  44842.0     84792.0
42294  2011-09-30  17244.0  17821.0  0.378000  0.390700  45618.0     83880.0
49114  2011-12-31  17167.0  17731.0  0.386600  0.399300  44401.0     85059.0
55903  2012-03-31  17302.0  17850.0  0.398900  0.411500  43376.0     86567.0
63280  2012-06-30  17287.0  17830.0  0.403300  0.416000  42860.0     84513.0
70594  2012-09-30  17363.0  17913.0  0.398400  0.411000  43582.0     82981.0
77843  2012-12-31  17425.0  17955.0  0.414200  0.426800  42073.0     84556.0
84993  2013-03-31  17513.0  18040.0  0.419500  0.432200  41743.0     86640.0
92078  2013-06-30  17626.0  18177.0  0.403400  0.416100  43689.0     88879.0
99083  2013-09-30  17836.0  18431.0  0.377800  0.390400  47213.0     86285.0
106038 2013-12-31  17992.0  18544.0  0.410900  0.423500  43783.0     86264.0
112915 2014-03-31  18201.0  18745.0  0.421900  0.434600  43136.0     85584.0
119709 2014-06-30  18329.0  18882.0  0.418200  0.430800  43830.0     84865.0
126431 2014-09-30  18590.0  19163.0  0.408700  0.421300  45484.0     84405.0
133082 2014-12-31  18696.0  19238.0  0.435800  0.448400  42904.0     86243.0
139652 2015-03-31  18711.0  19139.0  0.555519  0.568226  33682.0     89230.0
146132 2015-06-30  18970.0  19559.0  0.407300  0.419946  46575.0     88495.0
152540 2015-09-30  19768.0  20326.0  0.447280  0.459906  44196.0     87142.0
158868 2015-12-31  19156.0  19709.0  0.438092  0.450739  43726.0     87800.0
165106 2016-03-31  21798.0  22329.0  0.519111  0.531757  41991.0     88907.0
171282 2016-06-30  19821.0  20369.0  0.456505  0.469126  43419.0     87052.0
177394 2016-09-30  20070.0  20634.0  0.449506  0.462138  44649.0     85937.0
183428 2016-12-31  20231.0  20767.0  0.477248  0.489892  42391.0     85756.0
189394 2017-03-31  14428.0  14937.0  0.359102  0.371771  40178.0     85768.0
195302 2017-06-30  14472.0  14990.0  0.353251  0.365895  40968.0     84568.0
201141 2017-09-30  14739.0  15296.0  0.334415  0.347053  44074.0     81241.0
206930 2017-12-31  14625.0  15120.0  0.374443  0.387117  39058.0     81711.0
212651 2018-03-31  15235.0  15756.0  0.370258  0.382920  41147.0     81624.0
218308 2018-06-30  15272.0  15803.0  0.363221  0.375850  42046.0     79014.0
223901 2018-09-30  15535.0  16012.0  0.412255  0.424913  37683.0     75640.0
229428 2018-12-31  13884.0  14359.0  0.370843  0.383531  37439.0     74798.0
234884 2019-03-31  15545.0  16031.0  0.404512  0.417159  38429.0     74561.0
240295 2019-06-30  15931.0  16429.0  0.404566  0.417213  39378.0     75246.0
245647 2019-09-30  15606.0  16110.0  0.391628  0.404276  39849.0     74406.0